In [35]:
import pandas as pd
import numpy as np
import re
from math import ceil, floor
from random import randint
import random

In [36]:
csv_data = pd.read_csv('data.csv', sep=',')

In [37]:
csv_data

,imei 1/sn,sku_name,activate_time
0,867555052196438,Redmi 9A Sea Blue 3GB RAM 32GB ROM,10/10/2020 11:10
1,862851050440414,Redmi 9A Sea Blue 2GB RAM 32GB ROM,10/10/2020 11:32
2,862851050447211,Redmi 9A Sea Blue 2GB RAM 32GB ROM,10/10/2020 11:32
3,862851050251118,Redmi 9A Sea Blue 2GB RAM 32GB ROM,10/10/2020 12:19
4,867250052667162,Redmi 9 Prime Space Blue 4GB RAM 64GB ROM,10/10/2020 12:57
...,...,...,...
813,860868043367670,Redmi 9A Nature Green 2GB RAM 32GB ROM,11/16/2020 20:37
814,860868042982750,Redmi 9A Nature Green 2GB RAM 32GB ROM,11/16/2020 20:37
815,860868045679973,Redmi 9A Nature Green 2GB RAM 32GB ROM,11/16/2020 20:41
816,860868045677134,Redmi 9A Nature Green 2GB RAM 32GB ROM,11/16/2020 20:42


In [38]:
names = pd.read_csv('names.csv',sep=',')
names

,Unnamed: 0,Name
0,0,Tamu Banerjee
1,1,Kamlesh Thakur
2,2,Nisha Mandalik
3,3,Sushila Manak
4,4,Bhram Mohan
...,...,...
894,895,Rakhi Puri
895,896,Meena Lalwani
896,897,Deepa Mandalik
897,898,Badri Chowdhury


In [39]:
model_keywords = ["Redmi", "9A", "3GB", "32GB", "2GB","8GB", "4GB", "64GB", "Note", "9", "Pro", "Max", "8A",
                  "Dual","Mi", "A3", "6GB", "128GB", "Prime", "9i", "8", "10T", "Y3"]
def reduced_name(name):
    gen = []
    for key in model_keywords:
        if key in name.split(" "):
            gen.append(key)
    return " ".join(gen)

In [40]:
price_dict= {
    "Redmi 9A 3GB 32GB" : 7500,
    "Redmi 9A 2GB 32GB" : 7000,
    "Redmi 9 Prime 4GB 64GB": 10000,
    "Redmi 9 Prime 4GB 128GB": 12000,
    "Redmi Note 9 Pro Max 6GB 128GB": 18500,
    "Redmi Note 9 Pro Max 6GB 64GB": 17000,
    "Redmi 8A Dual 3GB 64GB":9000,
    "Redmi 8A Dual 3GB 32GB":8300,
    "Redmi 8A Dual 2GB 32GB":7500,
    "Mi A3 4GB 64GB": 13000,
    "Redmi 9 4GB 64GB": 9000,
    "Redmi Note 9 6GB 128GB": 15000,
    "Redmi 9i 4GB 128GB": 9300,
    "Redmi 9i 4GB 64GB": 8300,
    "Redmi Note 9 4GB 128GB": 13500,
    "Redmi Note 9 4GB 64GB": 12000,
    "Redmi Note 9 Pro 4GB 64GB": 14000,
    "Redmi Note 9 Pro 4GB 128GB": 16000,
    "Redmi Note 8 4GB 64GB": 12500,
    "Mi 10T Pro 8GB 128GB": 40000,
    "Redmi Y3 4GB 64GB": 12000,
    "Redmi 9 4GB 128GB": 10000,
}

def get_price(model):
    for model_dict_single in price_dict.keys():
        if set(model.split(" ")) == set(model_dict_single.split(" ")):
            return round(price_dict[model_dict_single]/(1+(18/100)),2)

def get_roundoff(num):
    ceiled = int(num)
    oneth = int(str(ceiled)[-1])
    if oneth == 5 or oneth == 0:
        return ceiled
    factor = 10 - oneth if oneth != 0 and oneth != 5 and oneth > 5 else (-1 * oneth)
    return ceiled + factor

def serial_no(start, stop, last=5750):
    return last + randint(start, stop)

In [41]:
from datetime import datetime
# noned = []
class Voucher:
    def __init__(self, name, array, index):
        self.index = index
        self.name = name
        self.serial = 0
        self.imei = array[0]
        self.model = reduced_name(array[1])
        self.price = float(get_price(self.model))
        self.cgst =  round(self.price * 9/100,2)
        self.sgst =  round(self.price * 9/100, 2)
        self.total_price = round(self.price + self.cgst + self.sgst,2)
        self.roundoff = round(float(get_roundoff(self.total_price)) - self.total_price,2)
        self.grand_total = round(self.total_price + self.roundoff,2)
        self.time = datetime.strptime(array[2], "%m/%d/%Y %H:%M")
    def get_time(self):
        return self.time.strftime("%d-%m-%Y")
    def getArray(self, serial):
        self.serial = serial
        return [self.serial,self.name, self.imei, self.model, self.get_time(), self.price, self.cgst, self.sgst, self.roundoff, self.total_price, self.grand_total]

In [42]:
voucher_list =[]
names_values = names.values
for index,data in enumerate(csv_data.values):
    voucher_list.append(Voucher(names_values[index][1], data, index))

In [43]:
voucher_data = []
for index, data in enumerate(voucher_list):
    voucher_data.append(data.getArray(serial_no(1,10, last=voucher_list[index -1].serial if index > 0 else 5750)))

In [44]:
frame = pd.DataFrame(voucher_data, columns=['Serial','Name', 'IMEI', 'Item', 'Time', 'Price','CGST', 'SGST','Round Off', 'Total', 'Grand Total'])

In [45]:
names = [name[1] for name in frame.values]
names
# frame.to_csv('bills.csv')

['Tamu Banerjee',
 'Kamlesh Thakur',
 'Nisha Mandalik',
 'Sushila Manak',
 'Bhram Mohan',
 'Reena Chandra',
 'Anupam Lahoti',
 'Rani Saoji',
 'Suraj Mandalik',
 'Bharat Jha',
 'Ved Mohan',
 'Aarti Sanon',
 'Rajan Mohan',
 'Fauji Banerjee',
 'Riya Chandra',
 'Madhu Chaturvedi',
 'Ashrabi Ashtikar',
 'Geeta Jaiswal',
 'Dharmender Asthana',
 'Keshav Bhan',
 'Faizur Bhatia',
 'Laxmi Mishra',
 'Baby Bharjatya',
 'Jyoti Mohan',
 'Harleen Tiwari',
 'Amit Varma',
 'Ashok Puranik',
 'Rajesh Pandey',
 'Asgari Luthra',
 'Naveen Banthiya',
 'Vishal Salmaan',
 'Anita Gautam',
 'Parmod Basu',
 'Preeti Bhalla',
 'Kavita Kulkarni',
 'Basant Jalan',
 'Pooja Chaurasia',
 'Naveen Asthana',
 'Hardeep Thapa',
 'Ghanshyam Marwah',
 'Geeta Patel',
 'Mohan Chaurasia',
 'Pooja Mallya',
 'Krishan Pandey',
 'Km. Bharadwaj',
 'Anuj Amarnath',
 'Reena Wadhawa',
 'Raja Chaurasia',
 'Rukiya Bhagwat',
 'Kuldeep Yadav',
 'Anand Jalan',
 'Ankush Malik',
 'Shalu Bhagwat',
 'Durga Wadhawa',
 'Shanta Joshi',
 'Anil Pandey

In [27]:
from fpdf import FPDF

In [46]:
from num2words import num2words
def int_to_en(num):
    return num2words(num).title()


In [47]:
class VoucherPDF:
    def __init__(self, voucher, pdf):
        self.voucher = voucher
        self.pdf = pdf
    def render(self):
        self.pdf.add_page()
        self.header()
        self.body()
        self.footer()
#         self.pdf.output('out.pdf')
    
    def header(self):
        ## Juridiction
        self.pdf.set_font('Arial', 'U', 8)
        self.pdf.cell(0,10, 'SUBJECTED TO GORAKHPUT JURISDICTION', 0, 1, 'C')
        
        ## Invoice No
        self.pdf.set_font('Arial', '', 8)
        self.pdf.cell(13,10,'Invoice No. ', 0,0 ,'C')
        self.pdf.set_font('Arial', 'B', 8)
        self.pdf.cell(8,10, str(self.voucher.serial), 0,0,'C')
#         self.pdf.cell(0,6,'',0,1,'C')
        ## Date
        self.pdf.set_font('Arial', '', 8)
        self.pdf.cell(86,10, '', 0,0, 'R')
        self.pdf.cell(12,10, 'Date ',0,0,'C')
        self.pdf.set_font('Arial', 'B', 8)
        self.pdf.cell(12,10, self.voucher.get_time(),0,0,'C')
        
        #Name of My Company
        self.pdf.cell(0,8,'',0,1,'C')
        self.pdf.set_font('Arial', 'B', 10)
        self.pdf.cell(0,4, 'Jaiswal Trading Company',0,1,'C')
        self.pdf.set_font('Arial', '', 8)
        self.pdf.cell(0,4, 'Shop No. 15 Baldev Plaza',0,1,'C')
        self.pdf.cell(0,4, 'Gorakhpur',0,1,'C')
        self.pdf.cell(0,4, f'GSTIN: 09AFCPJ3314B1Z3',0,1,'C')
        self.pdf.set_font('Arial', 'U', 8)
        self.pdf.cell(0,4, 'State Name: Uttar Pradesh, Code: 09',0,1,'C')
        
        # Tax Invoice
        self.pdf.cell(0,4,'',0,1,'C')
        self.pdf.set_font('Arial', 'B', 10)
        self.pdf.cell(0,4, 'Tax Invoice',0,1,'C')
        
        # Name
        self.pdf.cell(0,6,'',0,1,'C')
        self.pdf.set_font('Arial', 'B', 10)
        self.pdf.cell(0,4, self.voucher.name.encode('latin-1', 'replace').decode('latin-1') if not "?" in self.voucher.name else random.choice(["Krishna", "Adil", "Rama", "Rsihab"]) ,0,1,'C')
        self.pdf.set_font('Arial', '', 6)
        self.pdf.cell(0,4, 'Gorakhpur',0,1,'C')
        
    
    def body(self):
        self.pdf.cell(0,4,'', 0,1,'C')
        
        ### body header
        self.pdf.set_font('Arial', '', 6)
        self.pdf.cell(55,8,'Description of Goods', 1,0,'C')
        self.pdf.cell(20,8,'Quantity', 1,0,'C')
        self.pdf.cell(20,8,'Rate', 1,0,'C')
        self.pdf.cell(10,8,'per', 1,0,'C')
        self.pdf.cell(30,8,'Amount', 1,0,'C')
        
        self.pdf.cell(0,8,'',0,1,'C')
        self.pdf.line(10,78,10,120)
        
        # Item detail
        self.pdf.set_font('Arial', 'B', 8)
        self.pdf.cell(55,8, self.voucher.model,0,0,'L')
        self.pdf.line(65,78,65,120)
        self.pdf.cell(20,8,'1 pcs', 0,0,'C')
        self.pdf.line(85,78,85,120)
        self.pdf.cell(20,8,f'{self.voucher.price}', 0,0,'C')
        self.pdf.line(105,78,105,120)
        self.pdf.cell(10,8,'pcs', 0,0,'C')
        self.pdf.line(115,78,115,120)
        self.pdf.cell(30,8,f'{self.voucher.price}', 0,0,'R')
        self.pdf.line(145,78,145,120)
        self.pdf.cell(0,4,'',0,1,'C')
        self.pdf.set_font('Arial', 'BI', 8)
        self.pdf.cell(35,8, str(self.voucher.imei),0,1,'C')

        ## Taxes
        #CGST
        self.pdf.cell(0,8,'',0,1,'C')
        self.pdf.set_font('Arial', 'B', 8)
        self.pdf.cell(55,8, 'CGST',0,0,'R')
        self.pdf.cell(20,8,'', 0,0,'C')
        self.pdf.cell(20,8,'', 0,0,'C')
        self.pdf.cell(10,8,'', 0,0,'C')
        self.pdf.cell(30,8,f'{self.voucher.cgst}', 0,0,'R')
        
        #SGST
        self.pdf.cell(0,4,'',0,1,'C')
        self.pdf.set_font('Arial', 'B', 8)
        self.pdf.cell(55,8, 'SGST',0,0,'R')
        self.pdf.cell(20,8,'', 0,0,'C')
        self.pdf.cell(20,8,'', 0,0,'C')
        self.pdf.cell(10,8,'', 0,0,'C')
        self.pdf.cell(30,8,f'{self.voucher.sgst}', 0,0,'R')
        
        # RoundOFF
        self.pdf.cell(0,4,'',0,1,'C')
        self.pdf.set_font('Arial', 'B', 8)
        self.pdf.cell(55,8, 'Round off',0,0,'R')
        self.pdf.cell(20,8,'', 0,0,'C')
        self.pdf.cell(20,8,'', 0,0,'C')
        self.pdf.cell(10,8,'', 0,0,'C')
        self.pdf.cell(30,8,f'{self.voucher.roundoff}', 0,0,'R')
        
        ## Total
        self.pdf.cell(0,10,'',0,1,'C')
        self.pdf.set_font('Arial', '', 6)
        self.pdf.cell(55,6,'Total', 1,0,'R')
        self.pdf.cell(20,6,'1 pcs', 1,0,'C')
        self.pdf.cell(20,6,'', 1,0,'C')
        self.pdf.cell(10,6,'', 1,0,'C')
        self.pdf.set_font('Arial', 'B', 8)
        self.pdf.cell(30,6, f'{self.voucher.grand_total}', 1,0,'R')
        
#         self.pdf.cell(0,8,'',0,1,'C')
        self.pdf.line(10,78,10,120)
    
    def footer(self):
        self.pdf.cell(0,6,'',0,1,'C')
        self.pdf.set_font('Arial', '', 6)
        self.pdf.cell(55,6, 'Amount Chargeable(words)',0,0,'L')
        self.pdf.cell(20,6,'', 0,0,'C')
        self.pdf.cell(20,6,'', 0,0,'C')
        self.pdf.cell(10,6,'', 0,0,'C')
        self.pdf.cell(30,6,f'E & O.E', 0,1,'R')
        
        # In words
        self.pdf.set_font('Arial', 'B', 8)
        self.pdf.cell(0,4, f'{int_to_en(self.voucher.grand_total)}',0,1,'L')
        
        ## Declaration
        self.pdf.cell(0,10,'',0,1,'C')
        self.pdf.set_font('Arial', 'U', 8)
        self.pdf.cell(10,6,'Declaration', 0,0, 'L')
        self.pdf.set_font('Arial', 'B', 8)
        self.pdf.cell(0,6,'for Jaiswal Trading Company', 0,1, 'R')
        
        self.pdf.set_font('Arial', '', 8)
        self.pdf.cell(0,4, 'We declare that this invoice shows the', 0,1, 'L')
        self.pdf.set_font('Arial', '', 6)
        self.pdf.cell(0,4, 'actual price of goods described and that all particulars are true and correct.', 0,1, 'L')
        
        self.pdf.set_font('Arial', 'B', 6)
        self.pdf.cell(0,6,'Authorised Signatory', 0,1, 'R')
        
        self.pdf.cell(0,10,'',0,1,'C')
        self.pdf.set_font('Arial', 'U', 8)
        self.pdf.cell(0,10,'This is a Computer Generated Invoice',0,1,'C')
        
        
        
        
        

In [48]:

pdf = FPDF(orientation='P', format='A5')
for voucher in voucher_list:
    VoucherPDF(voucher, pdf).render()
pdf.output('out.pdf')

''